In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def get_features_for_test(data):
    # FE for data as row of DataFrame
    
    # Two new features from the competition tutorial
    #df_feat = data[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'timestamp']].copy()
    df_feat = data[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = df_feat['High'] - np.maximum(df_feat['Close'], df_feat['Open'])
    df_feat['Lower_Shadow'] = np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low']
    
    # Thanks to https://www.kaggle.com/code1110/gresearch-simple-lgb-starter
    df_feat['lower_shadow'] = np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low']
    df_feat['high2low'] = df_feat['High'] / df_feat['Low']
    df_feat['volume2count'] = df_feat['Volume'] / (df_feat['Count'] + 1)
    
    
    return df_feat.values

In [ ]:
import gresearch_crypto

In [ ]:
# Prediction and submission
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

from keras.models import load_model
import time
models = {}
for i in range(14):
    model_name = f'../input/gresearchcryptomodelv2/model-{i}.hdf5'
    #print(model_name)
    models[i] = load_model(model_name)

time_taken = []
n_steps = 12
for i, (df_test, df_pred) in enumerate(iter_test):
    for j, row in df_test.iterrows():
        
        try:
            #init_time = time.time()
            model = models[row['Asset_ID']]
            x_test = get_features_for_test(row)
            x_test = x_test.reshape((1, n_steps))
            y_pred = model.predict([x_test])[0]

            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
            #time_taken.append(time.time() - init_time)
        
        except:
            print(f'{i}-th iteration of the test dataset, {j}-th row - there was the exception, then set Target = 0')
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
            
#         # Print just one sample row to get a feeling of what it looks like        
#         if i == 0 and j == 0:
#             print('Example of the x_test data')
#             display(x_test)

#     # Display the first prediction dataframe
#     if i == 0:
#         print('Example of the prediction for test data')
#         display(df_pred)
    df_pred['Target'] = df_pred['Target'].fillna(0)

    # Send submissions
    env.predict(df_pred)

In [ ]:
!ls -ltr